In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. Re-define the exact same model architecture
class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_size, hidden_dim):
        super(CNNTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, n_filters, kernel_size=filter_size, padding='valid')
        self.pool1 = nn.MaxPool1d(kernel_size=3)
        self.conv2 = nn.Conv1d(n_filters, n_filters, kernel_size=filter_size, padding='valid')
        self.pool2 = nn.MaxPool1d(kernel_size=3)
        self.conv3 = nn.Conv1d(n_filters, n_filters, kernel_size=filter_size, padding='valid')
        self.global_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(n_filters, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids).permute(0, 2, 1)
        x = self.pool1(F.relu(self.conv1(embedded)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.global_pool(x).squeeze(2)
        x = F.relu(self.fc1(x))
        output = torch.sigmoid(self.fc2(x))
        return output

# 2. Instantiate the model with the same parameters used during training
model = CNNTextClassifier(
    vocab_size=30522,
    embedding_dim=50,
    n_filters=128,
    filter_size=3,
    hidden_dim=128
)

# 3. Load the saved weights
model.load_state_dict(torch.load("../cnn_model_trained_torch/cnn_model_trained_pytorch.pth"))

# 4. Set the model to evaluation mode
model.eval()
print("PyTorch model loaded and set to evaluation mode.")

PyTorch model loaded and set to evaluation mode.


In [3]:
# --- 3. Export to ONNX ---
print("\nExporting model to ONNX...")
dummy_input = torch.randint(0, 1000, (1, 128), dtype=torch.long)
onnx_model_path = "cnn_model.onnx"
torch.onnx.export(
    model, dummy_input, onnx_model_path,
    input_names=['input_ids'], output_names=['output'], opset_version=12
)
print(f"Model exported to {onnx_model_path}")


Exporting model to ONNX...
Model exported to cnn_model.onnx


In [4]:
import coremltools as ct

dummy_input = torch.randint(0, 1000, (1, 128), dtype=torch.long)

traced_model = torch.jit.trace(model, dummy_input)
exported_program = torch.export.export(model, (dummy_input, ))

model_from_trace = ct.convert(
    traced_model,
    inputs=[ct.TensorType(shape=dummy_input.shape)],
)
model_from_export = ct.convert(exported_program)

When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Running MIL default pipeline:   0%|          | 0/89 [00:00<?, ? passes/s]/Users/mahas/PycharmProjects/STIRS/.venv/lib/python3.9/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:273: UserWarning: Output, '72', of the source model, has been renamed to 'var_72' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1902.39 passes/s]
When both 'convert_to' and 'minimum_deployment_t

NotImplementedError: Conversion for models with only ATEN or EDGE dialect is supported/tested. Provided Dialect: TRAINING

In [5]:
model_from_trace.save("newmodel_from_trace.mlpackage")